In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from candlesticks import barplot

In [ ]:
nq = pd.read_pickle('minute_NQ_cont_non_active_included.pickle')
nq.head()

In [ ]:
nq.describe()

In [ ]:
nq.head()

In [ ]:
nq.close.plot(figsize=(20,8));

In [ ]:
data = pd.DataFrame(nq.close)
data['max'] = data.close.rolling(200).max()
data['min'] = data.close.rolling(200).min()
data['ema'] = data.close.rolling(2000).mean()
#data['mean'] = (data['max'] + data['min']) / 2
#data['indicator'] = (data['close'] - data['mean']) / (data['max'] - data['min'])

data.tail()

In [ ]:
data[-75000:].plot(figsize=(20,8), grid=True);

In [ ]:
avg_vol = nq.loc['20190621':].volume.rolling(30).sum().mean()
avg_vol

In [ ]:
time_int = 30
df = nq.loc['20190621':]
avg = df.volume.rolling(time_int).sum().mean()
volume_df = pd.DataFrame({
    '{}m'.format(time_int): df.volume.rolling(time_int).sum(),
    '{}m avg'.format(time_int): avg,
})
print(avg)
volume_df.plot(figsize=(20, 8), grid=True);

In [ ]:
volume_df['rolling_avg'] = volume_df.iloc[:,0].rolling(2400).mean()
volume_df.plot(figsize=(18,7), grid=True);

In [2]:
 nq.loc['20190611 22:45:00':'20190621'].close.plot(figsize=(20,10),grid=True);

NameError: name 'nq' is not defined

In [ ]:
class VolumeGrouper:
    def __init__(self, avg_vol):
        self.volume = 0
        self.counter = 0
        self.avg_vol = avg_vol
    def group(self, vol):
        self.volume += vol
        if self.volume >= self.avg_vol:
            self.volume -= self.avg_vol
            self.counter += 1
        return self.counter

In [ ]:
vol_candles = nq.copy().reset_index()
vol_candles['turnover'] = vol_candles.close * vol_candles.volume
grouper = VolumeGrouper(avg_vol)
vol_candles['label'] = vol_candles.volume.apply(grouper.group)

In [ ]:
vol_candles = vol_candles.groupby('label').agg({'date': 'last',
                                  'open': 'first',
                                 'high': 'max',
                                 'low': 'min',
                                 'close': 'last',
                                 'volume': 'sum',
                                'turnover': 'sum'})
vol_candles['avg_price'] = vol_candles.turnover / vol_candles.volume

In [ ]:
#vol_candles.set_index('date', inplace=True)

In [ ]:
vol_candles.head()

In [ ]:
vol_candles['close'] = vol_candles['avg_price']

In [ ]:
data = pd.DataFrame(vol_candles.close)
data['max'] = data.close.rolling(20).max()
data['min'] = data.close.rolling(20).min()
data['ema_fast'] = data.close.ewm(span=80).mean()
data['ema_slow'] = data.close.ewm(span=480).mean()
#data.reset_index(drop=True, inplace=True)
data.tail()

In [ ]:
data.plot(figsize=(18,7), grid=True);

In [ ]:
ema_diff = pd.DataFrame({'close': data['close'], 'ema_diff': data['ema_fast'] - data['ema_slow']})
ema_diff['return'] = ema_diff['close'].pct_change()
ema_diff['std'] = ema_diff['return'].expanding().std()
ema_diff['price_std'] = ema_diff['std'] * ema_diff['close']
ema_diff['norm'] = ema_diff['ema_diff']/ema_diff['price_std']
ema_diff['zscore'] = (ema_diff['ema_diff'] - ema_diff['ema_diff'].expanding().mean()) / ema_diff['ema_diff'].expanding().std()
#ema_diff['max']

In [ ]:
ema_diff.tail()

In [ ]:
ema_diff['norm'].plot(figsize=(18,7), grid=True);

In [ ]:
ema_diff['zscore'].plot(figsize=(18,7), grid=True);

In [ ]:
def get_forecast(close, periods):
    ind = pd.DataFrame({'close': close})
    ind['max'] = ind.close.rolling(periods).max()
    ind['min'] = ind.close.rolling(periods).min()
    ind['mean'] = (ind['max'] + ind['min']) / 2
    ind['forecast'] = (ind['close'] - ind['mean']) / (ind['max'] - ind['min'])
    return ind

In [ ]:
ind = get_forecast(data.close, 120)
ind.forecast.plot(figsize=(18,7), grid=True);

In [ ]:
10 / ind.forecast.abs().mean()

In [ ]:
10 / ema_diff.zscore.abs().mean()

In [ ]:
10 / ema_diff.norm.abs().mean()

In [ ]:
ind['ind'] = ind['forecast'] * 40 + ema_diff['norm'] * 5.6

In [ ]:
ind.ind.plot(figsize=(18, 7), grid=True);

In [ ]:
norm_ind = pd.DataFrame({'breakout': ind['forecast'] * 40, 'ema_cross': ema_diff['norm'] * 5.6})
norm_ind['ema_cross'] = norm_ind['ema_cross'].clip(lower=-20, upper=20)
norm_ind['ema_cross_shift'] = norm_ind['ema_cross'].shift(20)
norm_ind.tail()

In [ ]:
norm_ind.plot(figsize=(18,7), grid=True);

In [ ]:
(norm_ind['breakout'] - norm_ind['ema_cross_shift']).plot(figsize=(18,7), grid=True);

In [ ]:
data.plot(figsize=(18,7), grid=True);

In [ ]:
periods = [7, 15, 30, 120, 240, 480]

In [ ]:
forecasts = pd.DataFrame()
for period in periods:
    forecasts[period] = get_forecast(data.close, period)['forecast']
forecasts.tail()

In [ ]:
x = pd.DataFrame({'mean_15': forecasts.mean(axis=1).ewm(span=15).mean(),
                  'mean_40': forecasts.mean(axis=1).ewm(span=40).mean()
                  #'forecast': forecasts.mean(axis=1)
                 })
x.plot(figsize=(18,7),grid=True);

In [ ]:
forecasts.corr()

In [ ]:
def get_emas(close, periods):
    ind = pd.DataFrame({'close': close})
    ind['ema_fast'] = close.ewm(span=periods[0]).mean()
    ind['ema_slow'] = close.ewm(span=periods[1]).mean()
    ind['diff'] = ind['ema_fast'] - ind['ema_slow']
    return ind

In [ ]:
ema_pairs = [(60, 240), (120, 640), (360, 1120)]

In [ ]:
emas = pd.DataFrame()
for pair in ema_pairs:
    emas[pair] = get_emas(data.close, pair)['diff']
emas.tail()

In [ ]:
emas.corr()

In [ ]:
emas.mean(axis=1).plot(figsize=(18,7), grid=True);

In [ ]:
data.plot(figsize=(18,7), grid=True);

In [ ]:
10/ emas.mean(axis=1).abs().mean()

In [ ]:
indices = pd.DataFrame({'ema': (emas.mean(axis=1) * .23).clip(lower=-20, upper=20),
                       'break': forecasts.mean(axis=1).rolling(15).mean() * 40
                       })
indices.plot(figsize=(18,7), grid=True);

In [ ]:
indices.corr()

In [ ]:
indices.mean(axis=1).plot(figsize=(18,7), grid=True);

In [ ]:
((indices['ema'] - indices['ema'].expanding().mean()) / indices['ema'].expanding().std()).plot(figsize=(18,7), grid=True);

In [ ]:
(indices['ema'] + indices['break']).plot(figsize=(18,7), grid=True);

In [ ]:
data.plot(figsize=(18,7), grid=True);

In [ ]:
def get_ATR(data, periods):
    TR = pd.DataFrame({'A': (data['high'] - data['low']),
                       'B': (data['high'] - data['close']).abs(),
                       'C': (data['low'] - data['close']).abs()
                      })
    TR['TR'] = TR.max(axis=1)
    TR['ATR'] = TR['TR'].rolling(periods).mean()
    return TR.ATR

In [ ]:
get_ATR(vol_candles, 60).plot(figsize=(18,7), grid=True);

In [ ]:
periods = [10, 20, 40, 80, 160]

In [ ]:
multi_period = pd.DataFrame({'close': data.close})

In [ ]:
multi_period['max'] = multi_period.close - multi_period.close.shift(1).rolling(20).max()
multi_period['min'] = multi_period.close.shift(1).rolling(20).min() - multi_period.close
multi_period[['max', 'min']].plot(figsize=(18,7), grid=True);

In [ ]:
multi_period_max = pd.DataFrame({'close': data.close})
multi_period_min = pd.DataFrame({'close': data.close})
for period in periods:
    multi_period_max[period] = multi_period.close - multi_period.close.shift(1).rolling(period).max()
    multi_period_min[period] = multi_period.close.shift(1).rolling(20).min() - multi_period.close
multi_period_avg = pd.DataFrame({
    #'close': data.close,
    'max': multi_period_max.mean(axis=1).clip(upper=2200, lower=800),
    'min': multi_period_min.mean(axis=1).clip(upper=2200, lower=800),
})
multi_period_avg.tail()

In [ ]:
multi_period_avg.plot(figsize=(18,7), grid=True);

In [ ]:
(multi_period_avg['max'] - multi_period_avg['min']).plot(figsize=(18,7), grid=True);

In [ ]:
from functools import partial

def get_min_max(data, period):
    return pd.DataFrame({
        'max': (data - data.shift(1).rolling(period).max()) > 0,
        'min': (data.shift(1).rolling(period).min() - data) > 0
    })

def majority_function(data):
    return (0.5 + ((data.sum(axis=1) - 0.5) / data.count(axis=1))).round(0)

def get_min_max_df(data, periods, func=get_min_max):
    min_max_func = partial(func, data)
    mins = pd.DataFrame()
    maxs = pd.DataFrame()
    for period in periods:
        df = min_max_func(period)
        mins[period] = df['min']
        maxs[period] = df['max']
    return {'min': mins,
           'max': maxs}

def get_signals(data, periods, func=get_min_max_df):
    min_max = func(data, periods)
    #return min_max['min']
    
    return pd.DataFrame({
        'signal': majority_function(min_max['max']) - majority_function(min_max['min'])
    })
    

In [ ]:
get_signals(data.close, periods).tail()

In [ ]:
signals = pd.DataFrame({
    'close': data.close,
    'signal': get_signals(data.close, periods)['signal']
})

In [ ]:
signals.plot();

In [ ]:
signals['value'] = signals['close'] * signals['signal']

In [ ]:
signals.tail()

In [ ]:
signals['value'] = signals.value.map(lambda x: x if x != 0 else np.nan)

In [3]:
signals['buy'] = signals['value'].map(lambda x: x if x > 0 else np.nan)
signals['sell'] = signals['value'].map(lambda x: -x if x < 0 else np.nan)

NameError: name 'signals' is not defined

In [ ]:
signals[['close', 'buy', 'sell']].plot(figsize=(18, 7), grid=True);

In [ ]:
signals['cum_signal'] = signals.signal.expanding().apply(lambda x: x.sum().clip(min=-1, max=1), raw=False)

In [ ]:
signals['cum_signal'].plot()

In [ ]:
#signals['filter'] = np.sign(emas.mean(axis=1))
signals['filter'] = np.sign(data.close - data.ema_fast)
signals['atr'] = get_ATR(vol_candles, 80)

In [ ]:
number_of_trades = 0
entry = 0
for row in signals.itertuples():
    if row.Index == 0:
        continue
    # if there's no position signal is to be filtered
    if signals.loc[row.Index-1, 'cum_signal'] == 0 and row.filter * row.signal != 1:
        signals.loc[row.Index, 'cum_signal'] = 0
        continue
    else:
        signals.loc[row.Index, 'cum_signal'] = (signals.loc[row.Index-1, 'cum_signal'] + row.signal).clip(max=1, min=-1)
        
    # create stop loss
    if signals.loc[row.Index, 'cum_signal'] - signals.loc[row.Index-1, 'cum_signal'] != 0:
        sl = signals.loc[row.Index, 'close'] - signals.loc[row.Index, 'cum_signal'] * signals.loc[row.Index, 'atr'] * 1
        signals.loc[row.Index, 'sl'] = sl
    if not signals.loc[row.Index, 'sl']:
        signals.loc[row.Index, 'sl'] = signals.loc[row.Index-1, 'sl']
        
    # apply stop loss
    if signals.loc[row.Index, 'cum_signal'] == 1 and signals.loc[row.Index, 'close'] < sl:
        signals.loc[row.Index, 'cum_signal'] = 0
    elif signals.loc[row.Index, 'cum_signal'] == -1 and signals.loc[row.Index, 'close'] > sl:
        signals.loc[row.Index, 'cum_signal'] = 0
        
    # count trades
    if signals.loc[row.Index, 'cum_signal'] != signals.loc[row.Index-1, 'cum_signal']:
        number_of_trades += 1

In [ ]:
signals.tail()

In [ ]:
signals['value'] = signals['close'] * signals['cum_signal']
signals['value'] = signals.value.map(lambda x: x if x != 0 else np.nan)
signals['long'] = signals['value'].map(lambda x: x if x > 0 else np.nan)
signals['short'] = signals['value'].map(lambda x: -x if x < 0 else np.nan)

In [ ]:
signals[['close', 'short', 'long']].plot(figsize=(18, 12), grid=True, color=['k', 'r', 'g']);

In [ ]:
#signals.tail(200)

In [ ]:
signals.head()

In [ ]:
signals['long_pnl'] = signals.long.diff().fillna(0)
signals['short_pnl'] = - signals.short.diff().fillna(0)

In [ ]:
long_pnl = signals.long_pnl.sum()
short_pnl = signals.short_pnl.sum()
pnl = long_pnl + short_pnl
long_pnl, short_pnl, pnl

fast filter: 773 (980 with 60minute volumes)
slow filter: 404

In [ ]:
signals['pnl'] = signals['long_pnl'] + signals['short_pnl']
pnl_p = signals[signals['pnl']>0].pnl
pnl_m = signals[signals['pnl']<0].pnl
pnl_p.count(), pnl_m.count(), pnl_p.count()/(pnl_p.count()+pnl_m.count())

In [ ]:
pnl_p.sum(), pnl_m.sum(), pnl_p.sum()/ pnl_m.abs().sum()

In [ ]:
signals['date'] = vol_candles['date']

In [ ]:
signals.set_index('date', inplace=True)

In [ ]:
signals[['close', 'short', 'long']].plot(figsize=(18, 12), grid=True);

In [ ]:
signals.tail(30)

In [ ]:
interval = pd.Series(signals.index).diff().loc[10:]
interval.tail(20)

In [ ]:
interval.mean()

In [ ]:
daily = pd.DataFrame()
daily['close'] = signals['close'].resample('D').last()
daily['pnl'] = (signals['long_pnl'] + signals['short_pnl']).resample('D').sum()
daily.tail()

In [ ]:
daily['dollars'] = 0
daily.iloc[0,2] = 25000

In [ ]:
daily.head()

In [ ]:
multiplier = 20

In [ ]:
daily['dollars'] = 25000 + daily.pnl.cumsum() * multiplier

In [ ]:
daily.tail()

In [ ]:
daily['return'] = daily.dollars.pct_change()

In [ ]:
daily.dollars.plot(figsize=(20,5), grid=True);

In [ ]:
daily['return'].hist(bins=50,figsize=(18,7), grid=True);

In [ ]:
daily['dollars_diff'] = daily.dollars.diff()
daily.tail()

In [ ]:
daily['dollars_diff'].sort_values(ascending=True).head(20)

In [ ]:
daily['dollars_diff'].sort_values(ascending=False).head(20)

In [ ]:
small = 0
big = []
for i in daily[daily['dollars_diff'] < 0].reset_index().date.diff().eq(pd.Timedelta('1 D')).iteritems():
    if i[1]:
        small += 1
    else:
        if small > 0:
            big.append(small)
            small = 0
print(big)

In [ ]:
daily['dollars_diff'].hist(bins=20,figsize=(18,7), grid=True);

In [ ]:
daily.tail()

In [ ]:
from pyfolio.timeseries import perf_stats

In [ ]:
perf_stats(daily['return'])

In [ ]:
(daily['return'] + 1).product()

In [ ]:
number_of_trades

In [ ]:
spread = .25 * multiplier
commission = 1.18 + .85 
cost_of_trades = number_of_trades * (spread + commission)
cost_of_trades

In [ ]:
days = (daily.index[-1] - daily.index[0]).days #pd.to_datetime('20190613')).days
days

In [ ]:
trades_per_day = number_of_trades / days
trades_per_day

In [ ]:
pnl_in_dollars = pnl*multiplier

In [ ]:
cost_of_trades / (pnl_in_dollars)

In [ ]:
mean = daily['return'].mean() * 252
std = daily['return'].std() * (252**.5)
mean, std

In [ ]:
mean/std

In [ ]:
#daily.to_pickle('cl_return.pickle')